# Correlation Power Analysis

This notebook demonstrates the functionality of the `CPA` class, which implements Correlation Power Analysis side-channel attack. The attack is performed and evaluated on leakage traces simulated by the `LeakageTarget` class.

## Initialisation

In [ ]:
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.mod import Mod
from pyecsca.ec.point import Point, InfinityPoint
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.sca.attack.leakage_model import HammingWeight
from pyecsca.sca.target.leakage import LeakageTarget
from random import randint
from pyecsca.sca.attack.CPA import CPA
import holoviews as hv
import multiprocessing as mp
import matplotlib.pyplot as plt
from pyecsca.sca.trace import Trace
import numpy as np
import warnings
from scipy.stats import ConstantInputWarning

In [ ]:
warnings.filterwarnings("ignore", category=ConstantInputWarning)

In [ ]:
hv.extension("bokeh")

We first define the elliptic curve parameters we are going to be using for the demonstration.

In [ ]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
p = 0xd7d1247f
a = Mod(0xa4a44016, p)
b = Mod(0x73f76716, p)
n = 0xd7d2a475
h = 1
gx, gy, gz = Mod(0x54eed6d7, p), Mod(0x6f1e55ac, p), Mod(1, p)
generator = Point(coords, X=gx, Y=gy, Z=gz)
neutral = InfinityPoint(coords)

curve = EllipticCurve(model, coords, p, neutral, {"a": a, "b": b})
params = DomainParameters(curve, generator, n, h)

add = coords.formulas["add-2015-rcb"]
dbl = coords.formulas["dbl-2015-rcb"]
scl = coords.formulas["z"]

mult = LTRMultiplier(add, dbl, None)

We create and initialize an instance of the `LeakageTarget` class with the above elliptic curve parameters.

In [ ]:
target = LeakageTarget(model, coords, mult, HammingWeight())
target.set_params(params)

Using the attack, we will try to recover an 8-bit scalar used in the scalar multiplication operation.

In [ ]:
scalar_bit_length = 8
secret_scalar = 229

We will perform scalar multiplication on 10 000 random points on the curve and simulate 10 000 corresponding leakage traces, which we will then use to evaluate the attack. 

In [ ]:
num_of_traces = 500

In [ ]:
generated_points, generated_traces = target.simulate_scalar_mult_traces(num_of_traces, secret_scalar)

## Performing the attack

We create and initialize an instance of the `CPA` class containing the attack's implementation.

In [ ]:
mult.init(params, params.generator)
real_pub_key = mult.multiply(secret_scalar)
leakage_model = HammingWeight()

In [ ]:
cpa = CPA(generated_points, generated_traces, leakage_model, mult, params)

We perform the attack and test that the scalar we recovered is correct.

In [ ]:
recovered_scalar = cpa.perform(scalar_bit_length, real_pub_key)
print(recovered_scalar)
print(recovered_scalar == secret_scalar)

We can visualize the correlations calculated during the attack for correct and incorrect guess of the second bit of the scalar and correct guesses for fourth and sixth bit of the scalar to show where in the computation were the correlations stongest.

In [ ]:
# Correct difference of means when guessing 2. bit (guessed scalar 192 = 1100 0000)
cpa.plot_correlations(cpa.correlations['guess_one'][0])

In [ ]:
# Incorrect difference of means when guessing 2. bit (guessed scalar 128 = 1000 0000)
cpa.plot_correlations(cpa.correlations['guess_zero'][0])

In [ ]:
# Correct difference of means when guessing 4. bit (guessed scalar 224 = 1110 0000)
cpa.plot_correlations(cpa.correlations['guess_zero'][2])

In [ ]:
# Correct difference of means when guessing 6. bit (guessed scalar 229 = 1110 0100)
cpa.plot_correlations(cpa.correlations['guess_one'][-2])

## Evaluating the effectiveness of the attack

We will evaluate how the success rate of the attack increases with an increasing number of traces and subsequently evaluate how the success rate for a fixed number of traces changes with increasing noise. 

To have a convenient way of generating random subsets, we define the following function:

In [ ]:
def select_random_sample(size, points, traces):
    sample_points = []
    sample_traces = []
    selected_numbers = []
    for _ in range(size):
        num = randint(0, num_of_traces - 1)
        if num not in selected_numbers:
            selected_numbers.append(num)
        sample_points.append(points[num])
        sample_traces.append(traces[num])
    return sample_points, sample_traces

### Success rate with changing number of traces

We take subsets of different sizes and plot the success rate of the DPA corresponding to them. For each size, we perform the DPA 100 times on 100 generated subsets.

In [ ]:
def CPA_traces(points, traces, sample_size, leakage_model, scalar_length, pub_key, multiplier, domain_params):
    sample_points, sample_traces = select_random_sample(sample_size, points, traces)
    sample_cpa = CPA(sample_points, sample_traces, leakage_model, multiplier, domain_params)
    recovered = sample_cpa.perform(scalar_length, pub_key)
    return recovered == secret_scalar

In [ ]:
number_of_subsets = 17
successes_rates_traces = []
number_of_samples = [x * 2 for x in range(1, number_of_subsets)]
for num in number_of_samples:
    with mp.Pool() as pool:
        result_objects = [pool.apply_async(CPA_traces, args=(generated_points, generated_traces, num, leakage_model, scalar_bit_length, 
                                                             real_pub_key, mult, params)) for _ in range(100)]
        results = [result.get() for result in result_objects]
    successes_rates_traces.append(sum(results) / 100)
    print(f"done for {num}")

In [ ]:
print(successes_rates_traces)

We plot the results.

In [ ]:
plt.plot(number_of_samples, successes_rates_traces, color='black')
plt.ylim([0, 1])
plt.xlabel('number of traces')
plt.ylabel('CPA success rate')

### Success rate with fixed number of traces and changing noise

As we now know the the minimum amount of traces needed for 100% success rate, we generate 100 subsets with that amount of traces and perform CPA for each subset. We do this multiple times with different amount of noise.

In [ ]:
def normalize_trace(trace: Trace):
    min_val = np.min(trace.samples)
    max_val = np.max(trace.samples)
    scaled_samples = (trace.samples - min_val) / (max_val - min_val) 
    trace.samples = scaled_samples   

def CPA_noise(points, traces, sample_size, standard_deviation, leakage_model, scalar_length, pub_key, multiplier, domain_params):
    sample_points, sample_traces = select_random_sample(sample_size, points, traces)
    for trace in sample_traces:
        normalize_trace(trace)
    noise = np.random.normal(0, standard_deviation, sample_traces[0].samples.shape)
    for trace in sample_traces:
        trace.samples = trace.samples + noise
    sample_cpa = CPA(sample_points, sample_traces, leakage_model, multiplier, domain_params)
    recovered = sample_cpa.perform(scalar_length, pub_key)
    return recovered == secret_scalar

In [ ]:
number_of_subsets = 17
successes_rates_noise = []
sample_size = 50
noise_sds = [0.2 * x for x in range(number_of_subsets)] 
for sd in noise_sds:
    with mp.Pool() as pool:
        result_objects = [pool.apply_async(CPA_noise, args=(generated_points, generated_traces, sample_size, sd, leakage_model, scalar_bit_length, 
                                                             real_pub_key, mult, params)) for _ in range(100)]
        results = [result.get() for result in result_objects]
    successes_rates_noise.append(sum(results) / 100)
    print(f"done for {sd}")

We plot the results.

In [ ]:
print(successes_rates_noise)

In [ ]:
plt.plot(noise_sds[0:-1], successes_rates_noise[0:-1], color='black')
plt.ylim([0, 1])
plt.xlabel('Gaussian noise (standard deviation)')
plt.ylabel('CPA success rate')